<a name="summary"></a>
### Summary

<a name="libs"></a>
### Import libraries and packages

In [ ]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle

# ! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

<a name="dataset"></a>
### Import dataset 

In [ ]:
# # Upload data file (use to manually upload if below cell not working)
# from google.colab import files
# import pandas as pd
# import io

# uploaded = files.upload()

Saving AMZN_2.csv to AMZN_2.csv


In [48]:
# Import dataset
bist100 = pd.read_csv("./AMZN_2.csv")
bist100.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,03-01-2017,37.384998,37.896000,70422000,37.938000,37.683498,37.683498
1,04-01-2017,37.709999,37.919498,50210000,37.984001,37.859001,37.859001
2,05-01-2017,38.013000,38.077499,116602000,39.119999,39.022499,39.022499
3,06-01-2017,38.924000,39.118000,119724000,39.972000,39.799500,39.799500
4,09-01-2017,39.588501,39.900002,68922000,40.088501,39.846001,39.846001


<a name="cname"></a>
### Rename columns

In [ ]:
# Rename columns
bist100.rename(columns={"Date":"date","Open":"open","High":"high","Low":"low","Close":"close"}, inplace= True)
bist100.head()

,date,low,open,Volume,high,close,Adjusted Close
0,03-01-2017,37.384998,37.896000,70422000,37.938000,37.683498,37.683498
1,04-01-2017,37.709999,37.919498,50210000,37.984001,37.859001,37.859001
2,05-01-2017,38.013000,38.077499,116602000,39.119999,39.022499,39.022499
3,06-01-2017,38.924000,39.118000,119724000,39.972000,39.799500,39.799500
4,09-01-2017,39.588501,39.900002,68922000,40.088501,39.846001,39.846001


<a name="predata"></a>

### Preprocessing Data

<a name="nullna"></a>
### Checking null and na value

In [ ]:
# Checking null value
bist100.isnull().sum()

date              0
low               0
open              0
Volume            0
high              0
close             0
Adjusted Close    0
dtype: int64

In [ ]:
# Checking na value
bist100.isna().any()

date              False
low               False
open              False
Volume            False
high              False
close             False
Adjusted Close    False
dtype: bool

In [ ]:
bist100.dropna(inplace=True)
bist100.isna().any()

date              False
low               False
open              False
Volume            False
high              False
close             False
Adjusted Close    False
dtype: bool

<a name="coldt"></a>

### Checking datatype of each column

In [ ]:
# Checking Data type of each column
print("Date column data type: ", type(bist100['date'][0]))
print("Open column data type: ", type(bist100['open'][0]))
print("Close column data type: ", type(bist100['close'][0]))
print("High column data type: ", type(bist100['high'][0]))
print("Low column data type: ", type(bist100['low'][0]))

Date column data type:  <class 'str'>
Open column data type:  <class 'numpy.float64'>
Close column data type:  <class 'numpy.float64'>
High column data type:  <class 'numpy.float64'>
Low column data type:  <class 'numpy.float64'>


<a name="dateformat"></a>

### Convert date from string to date format

In [ ]:
# convert date field from string to Date format and make it index
# bist100['date'] = pd.to_datetime(bist100.date)
bist100['date'] = pd.to_datetime(bist100.date,format='%d-%m-%Y')
bist100.head()

,date,low,open,Volume,high,close,Adjusted Close
0,2017-01-03,37.384998,37.896000,70422000,37.938000,37.683498,37.683498
1,2017-01-04,37.709999,37.919498,50210000,37.984001,37.859001,37.859001
2,2017-01-05,38.013000,38.077499,116602000,39.119999,39.022499,39.022499
3,2017-01-06,38.924000,39.118000,119724000,39.972000,39.799500,39.799500
4,2017-01-09,39.588501,39.900002,68922000,40.088501,39.846001,39.846001


<a name="sortdate"></a>

### Sorting dataset by date format

In [ ]:
bist100.sort_values(by='date', inplace=True)
bist100.head()

,date,low,open,Volume,high,close,Adjusted Close
0,2017-01-03,37.384998,37.896000,70422000,37.938000,37.683498,37.683498
1,2017-01-04,37.709999,37.919498,50210000,37.984001,37.859001,37.859001
2,2017-01-05,38.013000,38.077499,116602000,39.119999,39.022499,39.022499
3,2017-01-06,38.924000,39.118000,119724000,39.972000,39.799500,39.799500
4,2017-01-09,39.588501,39.900002,68922000,40.088501,39.846001,39.846001


In [ ]:
bist100.shape

(1497, 7)

<a name="eda"></a>

### EDA - Exploratory Data Analysis

<a name="duration"></a>

### Get the duration of dataset

In [ ]:
print("Starting date: ",bist100.iloc[0][0])
print("Ending date: ", bist100.iloc[-1][0])
print("Duration: ", bist100.iloc[-1][0]-bist100.iloc[0][0])

Starting date:  2017-01-03 00:00:00
Ending date:  2022-12-12 00:00:00
Duration:  2169 days 00:00:00


<a name="month_op_close"></a>

### Monthwise comparision between Stock actual, open and close price

In [ ]:
monthvise= bist100.groupby(bist100['date'].dt.strftime('%B'))[['open','close']].mean().sort_values(by='close')
monthvise.head()

,open,close
date,,
January,98.648816,98.623877
May,101.378598,101.346339
March,101.834440,101.841237
February,102.623518,102.589715
June,106.815390,106.839015


In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock actual, open and close price')
fig.show()

<a name="month_high_low"></a>

### Monthwise High and Low stock price 

In [ ]:
bist100.groupby(bist100['date'].dt.strftime('%B'))['low'].min()

date
April        44.224499
August       46.816502
December     56.237000
February     40.150002
January      37.384998
July         47.549999
June         46.349998
March        41.674999
May          46.389999
November     54.343498
October      47.518501
September    46.587502
Name: low, dtype: float64

In [ ]:
monthvise_high= bist100.groupby(bist100['date'].dt.strftime('%B'))['high'].max()
monthvise_low= bist100.groupby(bist100['date'].dt.strftime('%B'))['low'].min()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

<a name="trend"></a>

### Trend comparision between stock price, open price, close price, high price, low price

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(bist100, x=bist100.date, y=[bist100['open'], bist100['close'], 
                                          bist100['high'], bist100['low']],
             labels={'date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

<a name="closepred"></a>

### Close price prediction preparation and preprocessing

<a name="sepclose"></a>

### Make separate dataframe with close price

In [ ]:
closedf = bist100[['date','close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (1497, 2)


<a name="plotclose"></a>

### Plotting stock close price chart

In [ ]:
fig = px.line(closedf, x=closedf.date, y=closedf.close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.6)
fig.update_layout(title_text='Stock close price chart', plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="norm"></a>

### Normalizing / scaling close value between 0 to 1

In [ ]:
close_stock = closedf.copy()
del closedf['date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(1497, 1)


<a name="splitdata"></a>

### Split data for training and testing
Ratio for training and testing data is 65:35

In [ ]:
training_size=int(len(closedf)*0.65)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
train_data_reshape = train_data.reshape(-1)
test_data_reshape = test_data.reshape(-1)
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)
print("train_data_reshape: ", train_data_reshape.shape)
print("test_data_reshape: ", test_data_reshape.shape)

train_data:  (973, 1)
test_data:  (524, 1)
train_data_reshape:  (973,)
test_data_reshape:  (524,)


<a name="tsp"></a>

### Create new dataset according to requirement of time-series prediction 

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (957, 15)
y_train:  (957,)
X_test:  (508, 15)
y_test (508,)


### Initialize Evaluation Metric Lists

In [ ]:
# SVR, RF, SVR + RF, KNN, ARIMA, SARIMA, GRU + ARIMA, LSTM + ARIMA, LSTM, GRU, LSTM + GRU
num_models = 11

#RMSE, MSE, MAE

RMSE_Train = []
MSE_Train = []
MAE_Train = []
RMSE_Test = []
MSE_Test = []
MAE_Test = []

# explained variance
train_ev = []
test_ev = []

# R2 Score

R2_Train = []
R2_Test = []

# MGD, MPD

MGD_Train = []
MGD_Test = []

MPD_Train = []
MPD_Test = []


<a name="algo"></a>

## Algorithms

<a name="svr"></a>

### Super vector regression - SVR

In [ ]:
from sklearn.svm import SVR

svr_rbf = SVR(kernel= 'rbf', C= 1e2, gamma= 0.1)
svr_rbf.fit(X_train, y_train)

SVR(C=100.0, gamma=0.1)

In [ ]:
# Lets Do the prediction 

train_predict=svr_rbf.predict(X_train)
test_predict=svr_rbf.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)


Train data prediction: (957, 1)
Test data prediction: (508, 1)


In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="svrevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE

RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
MSE_Test.append(mean_squared_error(original_ytest,test_predict))
MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  6.396151196361093
Train data MSE:  40.91075012671144
Test data MAE:  4.9349600925875325
-------------------------------------------------------------------------------------
Test data RMSE:  8.62121192472498
Test data MSE:  74.3252950510202
Test data MAE:  6.758134014532531


<a name="svrevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9713300309911592
Test data explained variance regression score: 0.8830926570226498


<a name="svrrsquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9589344865645784
Test data R2 score: 0.8825236170169087


<a name="svrrloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.01049616182236936
Test data MGD:  0.003879982334101113
----------------------------------------------------------------------
Train data MPD:  0.6270522921941775
Test data MPD:  0.528673076768155


<a name="svrcomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="svrpred10"></a>

#### Predicting next 10 days

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        
        yhat = svr_rbf.predict(x_input)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = svr_rbf.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


<a name="svrlast25"></a>

#### Plotting last 15 days and next predicted 10 days

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="svrwholepred"></a>

#### Plotting whole closing stock price with prediction

In [ ]:
svrdf=closedf.tolist()
svrdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
svrdf=scaler.inverse_transform(svrdf).reshape(1,-1).tolist()[0]

names = cycle(['Close Price'])

fig = px.line(svrdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="rf"></a>

### Random Forest Regressor - RF

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [ ]:
# Lets Do the prediction 

train_predict=regressor.predict(X_train)
test_predict=regressor.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)


Train data prediction: (957, 1)
Test data prediction: (508, 1)


In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="rfevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
MSE_Test.append(mean_squared_error(original_ytest,test_predict))
MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  0.7807095843938214
Train data MSE:  0.6095074551643734
Test data MAE:  0.5084457758111793
-------------------------------------------------------------------------------------
Test data RMSE:  6.911474692499186
Test data MSE:  47.76848242505672
Test data MAE:  4.862012644106501


<a name="rfevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9993882860610223
Test data explained variance regression score: 0.9245617763117053


<a name="rfrsquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9993881868088089
Test data R2 score: 0.9244985367090043


<a name="rfrloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  6.096115021866507e-05
Test data MGD:  0.0023955759443575744
----------------------------------------------------------------------
Train data MPD:  0.005757927687772985
Test data MPD:  0.3348031879758061


<a name="rfcomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="rfpred10"></a>

#### Predicting next 10 days

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        
        yhat = regressor.predict(x_input)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = regressor.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


<a name="rflast25"></a>

#### Plotting last 15 days and next predicted 10 days

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="rfwholepred"></a>

#### Plotting whole closing stock price with prediction

In [ ]:
rfdf=closedf.tolist()
rfdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
rfdf=scaler.inverse_transform(rfdf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(rfdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

### SVR + RF

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor

# Instantiate the individual regressors
rf = RandomForestRegressor(n_estimators = 100, random_state = 0)
svr_rbf = SVR(kernel= 'rbf', C= 1e2, gamma= 0.1)

# Instantiate the ensemble model
ensemble = VotingRegressor(estimators=[('rf', rf), ('svr', svr_rbf)])

# Train the ensemble model
ensemble.fit(X_train, y_train)

VotingRegressor(estimators=[('rf', RandomForestRegressor(random_state=0)),
                            ('svr', SVR(C=100.0, gamma=0.1))])

In [ ]:
# Lets Do the prediction

train_predict=ensemble.predict(X_train)
test_predict=ensemble.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (957, 1)
Test data prediction: (508, 1)


In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [ ]:
# Evaluation metrices RMSE and MAE
#RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
#MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
#MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

#RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
#MSE_Test.append(mean_squared_error(original_ytest,test_predict))
#MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  3.333577551567551
Train data MSE:  11.112739292315108
Train data MAE:  2.586193950814241
-------------------------------------------------------------------------------------
Test data RMSE:  7.356576061245299
Test data MSE:  54.1192113448874
Test data MAE:  5.484966800178278


<a name="knnevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9919616704048746
Test data explained variance regression score: 0.9145239809097556


<a name="knnrsquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9888452217742411
Test data R2 score: 0.9144607607096532


<a name="knnrloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.00306648107539156
Test data MGD:  0.002736685406098113
----------------------------------------------------------------------
Train data MPD:  0.17443032872375508
Test data MPD:  0.379815244832886


<a name="knncomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [49]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Original close price vs Predicted close price (RF)',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="knnpred10"></a>

#### Predicting next 10 days

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        
        yhat = ensemble.predict(x_input)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = ensemble.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


<a name="knnlast25"></a>

#### Plotting last 15 days and next predicted 10 days

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="knnwholepred"></a>

#### Plotting whole closing stock price with prediction

In [ ]:
knndf=closedf.tolist()
knndf.extend((np.array(lst_output).reshape(-1,1)).tolist())
knndf=scaler.inverse_transform(knndf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(knndf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="knn"></a>

### K-nearest neighgbour - KNN

In [ ]:
from sklearn import neighbors

K = time_step
neighbor = neighbors.KNeighborsRegressor(n_neighbors = K)
neighbor.fit(X_train, y_train)


KNeighborsRegressor(n_neighbors=15)

In [ ]:
# Lets Do the prediction 

train_predict=neighbor.predict(X_train)
test_predict=neighbor.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)


Train data prediction: (957, 1)
Test data prediction: (508, 1)


In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="knnevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
MSE_Test.append(mean_squared_error(original_ytest,test_predict))
MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.4643188619547325
Train data MSE:  6.072867453385869
Test data MAE:  1.6150384879818878
-------------------------------------------------------------------------------------
Test data RMSE:  9.72230722448106
Test data MSE:  94.52325776719661
Test data MAE:  7.5885011683136465


<a name="knnevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9939047369990288
Test data explained variance regression score: 0.8510124360842222


<a name="knnrsquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9939041591946826
Test data R2 score: 0.8505993091228741


<a name="knnrloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0006164306199786173
Test data MGD:  0.004626704804687355
----------------------------------------------------------------------
Train data MPD:  0.05784911527975866
Test data MPD:  0.6530217720962477


<a name="knncomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="knnpred10"></a>

#### Predicting next 10 days

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        
        yhat = neighbor.predict(x_input)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = neighbor.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


<a name="knnlast25"></a>

#### Plotting last 15 days and next predicted 10 days

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="knnwholepred"></a>

#### Plotting whole closing stock price with prediction

In [ ]:
knndf=closedf.tolist()
knndf.extend((np.array(lst_output).reshape(-1,1)).tolist())
knndf=scaler.inverse_transform(knndf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(knndf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="ARIMA"></a>

### ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import copy
import warnings
train_predict = []
test_predict = []

with warnings.catch_warnings(record=True):
    for i in range(len(X_train)):
        model = ARIMA(X_train[i], order=(1, 1, 1))
        model_fit = model.fit()
        predictions = model_fit.forecast(steps=1)[0]
        # print(predictions)
        train_predict.append([predictions])

    for i in range(len(X_test)):
        model = ARIMA(X_test[i], order=(1, 1, 1))
        model_fit = model.fit()
        predictions = model_fit.forecast(steps=1)[0]
        # print(predictions)
        test_predict.append([predictions])

    # Another way to implement
    # train_data_reshape_dup = list(train_data_reshape.copy())
    # for i in range(len(test_data_reshape)):
    #     model = ARIMA(train_data_reshape_dup, order=(1, 1, 1))
    #     model_fit = model.fit()
    #     predictions = model_fit.forecast(steps=15)[-1]
    #     # print(predictions)
    #     test_predict.append([predictions])
    #     train_data_reshape_dup.append(test_data_reshape[i])
    # train_predict = list(train_data_reshape.reshape(-1,1))[16:]
    # train_predict = np.array(train_predict)

print(len(train_predict))
print(len(test_predict))


957
508


In [ ]:
# test_predict = test_predict[14:-2]
# test_predict = test_predict[:-16]
# print(len(test_predict))

In [ ]:
# for i in range(len(X_train)):
#     predictions = model_fit.forecast(steps=1)[0]
#     train_predict.append([predictions])

# train_predict = list(train_data_reshape.reshape(-1,1))[16:]
# test_predict = model_fit.forecast(steps=(len(test_data_reshape)))[16:]

train_predict = np.array(train_predict).reshape(-1,1)
test_predict = np.array(test_predict).reshape(-1,1)
print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (957, 1)
Test data prediction: (508, 1)


In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="knnevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data..

In [ ]:
# Evaluation metrices RMSE and MAE
RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
MSE_Test.append(mean_squared_error(original_ytest,test_predict))
MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.1837192718530325
Train data MSE:  4.768629858262338
Test data MAE:  1.371088570402539
-------------------------------------------------------------------------------------
Test data RMSE:  3.5974891853500224
Test data MSE:  12.941928438710367
Test data MAE:  2.5954411293909283


<a name="knnevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9952302407976003
Test data explained variance regression score: 0.979582232220723


<a name="knnrsquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

<a name="knnrloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.00045515312895445977
Test data MGD:  0.0007061521567174174
----------------------------------------------------------------------
Train data MPD:  0.04406541777095695
Test data MPD:  0.09391061137121688


<a name="knncomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="knnpred10"></a>

#### Predicting next 10 days

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10

with warnings.catch_warnings(record=True):
    while(i<pred_days):
        x_input=temp_input
        model = ARIMA(x_input, order=(1, 1, 1))
        model_fit = model.fit()
        predictions = model_fit.forecast(steps=1)[0]
        temp_input.append(predictions)
        temp_input=temp_input[1:]
        lst_output.append(predictions)
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


<a name="knnlast25"></a>

#### Plotting last 15 days and next predicted 10 days

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="knnwholepred"></a>

#### Plotting whole closing stock price with prediction

In [ ]:
arimadf=closedf.tolist()
arimadf.extend((np.array(lst_output).reshape(-1,1)).tolist())
arimadf=scaler.inverse_transform(arimadf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(arimadf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="SARIMA"></a>

### SARIMA

In [ ]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import copy
# test_predict = []
# train_data_reshape_dup = list(train_data_reshape.copy())
# model = sm.tsa.statespace.SARIMAX(train_data_reshape_dup, order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))
# results = model.fit()
# start_date = 1
# end_date = 88
# prediction = results.predict(start_date, end_date)
# print('111111111', prediction)
import warnings

train_predict = []
test_predict = []

with warnings.catch_warnings(record=True):
    for i in range(len(X_train)):
        model = sm.tsa.statespace.SARIMAX(X_train[i], order=(1, 1, 1), seasonal_order=(0, 1, 1, 2))
        model_fit = model.fit(disp=False)
        predictions = model_fit.forecast(steps=1)[0]
        # print(predictions)
        train_predict.append([predictions])

    for i in range(len(X_test)):
        model = sm.tsa.statespace.SARIMAX(X_test[i], order=(1, 1, 1), seasonal_order=(0, 1, 1, 2))
        model_fit = model.fit(disp=False)
        predictions = model_fit.forecast(steps=1)[0]
        # print(predictions)
        test_predict.append([predictions])

print(len(train_predict))
print(len(test_predict))



957
508


In [ ]:
train_predict = np.array(train_predict).reshape(-1,1)
test_predict = np.array(test_predict).reshape(-1,1)
print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (957, 1)
Test data prediction: (508, 1)


In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="knnevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data..

In [ ]:
# Evaluation metrices RMSE and MAE
RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
MSE_Test.append(mean_squared_error(original_ytest,test_predict))
MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.5275353860529775
Train data MSE:  6.388435127749974
Test data MAE:  1.5981650865003294
-------------------------------------------------------------------------------------
Test data RMSE:  4.406223800035478
Test data MSE:  19.414808175999088
Test data MAE:  3.128677483088571


<a name="knnevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9935879542563787
Test data explained variance regression score: 0.9693254371187389


<a name="knnrsquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

<a name="knnrloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0006314434956878162
Test data MGD:  0.001090998461834388
----------------------------------------------------------------------
Train data MPD:  0.059792073268815235
Test data MPD:  0.1430753305013034


<a name="knncomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="knnpred10"></a>

#### Predicting next 10 days

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
with warnings.catch_warnings(record=True):
    while(i<pred_days):
        x_input=temp_input
        model = sm.tsa.statespace.SARIMAX(x_input, order=(1, 1, 1), seasonal_order=(0, 1, 1, 2), verbose=0)
        model_fit = model.fit(disp=False)
        predictions = model_fit.forecast(steps=1)[0]
        temp_input.append(predictions)
        temp_input=temp_input[1:]
        lst_output.append(predictions)
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


<a name="knnlast25"></a>

#### Plotting last 15 days and next predicted 10 days

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="knnwholepred"></a>

#### Plotting whole closing stock price with prediction

In [ ]:
Sarimadf=closedf.tolist()
Sarimadf.extend((np.array(lst_output).reshape(-1,1)).tolist())
Sarimadf=scaler.inverse_transform(Sarimadf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(Sarimadf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="GRU + arima"></a>

### GRU + ARIMA

In [ ]:
# reshape input to be [samples, time steps, features] which is required for ARIMA
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1])

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (957, 15)
X_test:  (508, 15)


In [ ]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import copy
import warnings

arima_train_predict = []
arima_test_predict = []

with warnings.catch_warnings(record=True):
    for i in range(len(X_train)):
        model = sm.tsa.statespace.SARIMAX(X_train[i], order=(1, 1, 1), seasonal_order=(0, 1, 1, 2))
        model_fit = model.fit(disp=False)
        predictions = model_fit.forecast(steps=1)[0]
        # print(predictions)
        arima_train_predict.append([predictions])

    for i in range(len(X_test)):
        model = sm.tsa.statespace.SARIMAX(X_test[i], order=(1, 1, 1), seasonal_order=(0, 1, 1, 2))
        model_fit = model.fit(disp=False)
        predictions = model_fit.forecast(steps=1)[0]
        # print(predictions)
        arima_test_predict.append([predictions])

print(len(arima_train_predict))
print(len(arima_test_predict))

957
508


In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (957, 15, 1)
X_test:  (508, 15, 1)


In [ ]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(GRU(32,return_sequences=True,input_shape=(time_step,1)))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 15, 32)            3360      
                                                                 
 gru_1 (GRU)                 (None, 15, 32)            6336      
                                                                 
 gru_2 (GRU)                 (None, 15, 32)            6336      
                                                                 
 gru_3 (GRU)                 (None, 32)                6336      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 22,401
Trainable params: 22,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=5,verbose=1)

Epoch 1/100
192/192 [==============================] - 13s 32ms/step - loss: 0.0069 - val_loss: 0.0051
Epoch 2/100
192/192 [==============================] - 4s 22ms/step - loss: 8.2889e-04 - val_loss: 0.0020
Epoch 3/100
192/192 [==============================] - 5s 25ms/step - loss: 7.1041e-04 - val_loss: 0.0020
Epoch 4/100
192/192 [==============================] - 4s 23ms/step - loss: 7.0271e-04 - val_loss: 0.0013
Epoch 5/100
192/192 [==============================] - 4s 22ms/step - loss: 5.5960e-04 - val_loss: 0.0039
Epoch 6/100
192/192 [==============================] - 5s 25ms/step - loss: 5.9518e-04 - val_loss: 0.0010
Epoch 7/100
192/192 [==============================] - 4s 22ms/step - loss: 3.8993e-04 - val_loss: 7.5688e-04
Epoch 8/100
192/192 [==============================] - 5s 24ms/step - loss: 5.1266e-04 - val_loss: 0.0017
Epoch 9/100
192/192 [==============================] - 4s 23ms/step - loss: 4.0372e-04 - val_loss: 0.0020
Epoch 10/100
192/192 [=======================

In [ ]:
### Lets Do the prediction and check performance metrics
gru_train_predict=model.predict(X_train)
gru_test_predict=model.predict(X_test)
gru_train_predict.shape, gru_test_predict.shape

16/16 [==============================] - 0s 5ms/step


((957, 1), (508, 1))

In [ ]:
gru_train_predict = gru_train_predict.reshape(-1)
gru_test_predict = gru_test_predict.reshape(-1)
gru_test_predict = list(gru_test_predict)
gru_train_predict = list(gru_train_predict)
train_predict = []
test_predict = []
for i in range(len(gru_train_predict)):
    train_ele = (arima_train_predict[i] + gru_train_predict[i])/2
    train_predict.append(train_ele)

for i in range(len(gru_test_predict)):
    test_ele = (arima_test_predict[i] + gru_test_predict[i])/2
    test_predict.append(test_ele)
    
train_predict = np.array(train_predict)
test_predict = np.array(test_predict)

# # Transform back to original form
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="lstmevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
MSE_Test.append(mean_squared_error(original_ytest,test_predict))
MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.470485174761743
Train data MSE:  6.103296998717559
Test data MAE:  1.7684240861153782
-------------------------------------------------------------------------------------
Test data RMSE:  4.121320370025461
Test data MSE:  16.985281592386798
Test data MAE:  3.1064442215612136


<a name="knnevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9952840008145694
Test data explained variance regression score: 0.9786518490801455


<a name="knnrsquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9938736145359124
Test data R2 score: 0.9731535617308593


<a name="knnrloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0006429619108580854
Test data MGD:  0.0008921450741221891
----------------------------------------------------------------------
Train data MPD:  0.058734198851706505
Test data MPD:  0.12116289917897946


<a name="arima+lstm comparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="lstm + arima"></a>

### LSTM + ARIMA

In [ ]:
# reshape input to be [samples, time steps, features] which is required for ARIMA
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1])

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (957, 15)
X_test:  (508, 15)


In [ ]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import copy
import warnings

arima_train_predict = []
arima_test_predict = []

with warnings.catch_warnings(record=True):
    for i in range(len(X_train)):
        model = sm.tsa.statespace.SARIMAX(X_train[i], order=(1, 1, 1), seasonal_order=(0, 1, 1, 2))
        model_fit = model.fit(disp=False)
        predictions = model_fit.forecast(steps=1)[0]
        # print(predictions)
        arima_train_predict.append([predictions])

    for i in range(len(X_test)):
        model = sm.tsa.statespace.SARIMAX(X_test[i], order=(1, 1, 1), seasonal_order=(0, 1, 1, 2))
        model_fit = model.fit(disp=False)
        predictions = model_fit.forecast(steps=1)[0]
        # print(predictions)
        arima_test_predict.append([predictions])

print(len(arima_train_predict))
print(len(arima_test_predict))

957
508


In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (957, 15, 1)
X_test:  (508, 15, 1)


In [ ]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 21,025
Trainable params: 21,025
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=5,verbose=1)

Epoch 1/100
192/192 [==============================] - 9s 21ms/step - loss: 0.0078 - val_loss: 0.0058
Epoch 2/100
192/192 [==============================] - 3s 18ms/step - loss: 0.0014 - val_loss: 0.0061
Epoch 3/100
192/192 [==============================] - 4s 19ms/step - loss: 0.0013 - val_loss: 0.0048
Epoch 4/100
192/192 [==============================] - 3s 18ms/step - loss: 0.0012 - val_loss: 0.0040
Epoch 5/100
192/192 [==============================] - 3s 16ms/step - loss: 0.0012 - val_loss: 0.0035
Epoch 6/100
192/192 [==============================] - 3s 17ms/step - loss: 0.0011 - val_loss: 0.0051
Epoch 7/100
192/192 [==============================] - 4s 19ms/step - loss: 9.4819e-04 - val_loss: 0.0028
Epoch 8/100
192/192 [==============================] - 3s 16ms/step - loss: 9.2188e-04 - val_loss: 0.0046
Epoch 9/100
192/192 [==============================] - 4s 20ms/step - loss: 9.7258e-04 - val_loss: 0.0023
Epoch 10/100
192/192 [==============================] - 4s 19ms/step -

In [ ]:
### Lets Do the prediction and check performance metrics
lstm_train_predict=model.predict(X_train)
lstm_test_predict=model.predict(X_test)
lstm_train_predict.shape, lstm_test_predict.shape

16/16 [==============================] - 0s 7ms/step


((957, 1), (508, 1))

In [ ]:
lstm_train_predict = lstm_train_predict.reshape(-1)
lstm_test_predict = lstm_test_predict.reshape(-1)
lstm_test_predict = list(lstm_test_predict)
lstm_train_predict = list(lstm_train_predict)
train_predict = []
test_predict = []
for i in range(len(lstm_train_predict)):
    train_ele = (arima_train_predict[i] + lstm_train_predict[i])/2
    train_predict.append(train_ele)

for i in range(len(lstm_test_predict)):
    test_ele = (arima_test_predict[i] + lstm_test_predict[i])/2
    test_predict.append(test_ele)
    
train_predict = np.array(train_predict)
test_predict = np.array(test_predict)

# # Transform back to original form
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="lstmevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
MSE_Test.append(mean_squared_error(original_ytest,test_predict))
MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.102334361652026
Train data MSE:  4.419809768182832
Test data MAE:  1.3772150037664017
-------------------------------------------------------------------------------------
Test data RMSE:  3.7718968875861774
Test data MSE:  14.227206130582292
Test data MAE:  2.774199581769964


<a name="knnevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9956163100263348
Test data explained variance regression score: 0.9775367741810437


<a name="knnrsquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9955634703139111
Test data R2 score: 0.9775128949703008


<a name="knnrloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0004542102117823694
Test data MGD:  0.0007669059830576896
----------------------------------------------------------------------
Train data MPD:  0.042230314772681465
Test data MPD:  0.10275463487147977


<a name="arima+lstm comparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="lstm"></a>

### LSTM

In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (957, 15, 1)
X_test:  (508, 15, 1)


<a name="lstmevalmat"></a>

#### LSTM model structure

In [ ]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 21,025
Trainable params: 21,025
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=5,verbose=1)

Epoch 1/100
192/192 [==============================] - 9s 21ms/step - loss: 0.0067 - val_loss: 0.0057
Epoch 2/100
192/192 [==============================] - 3s 17ms/step - loss: 0.0016 - val_loss: 0.0070
Epoch 3/100
192/192 [==============================] - 3s 18ms/step - loss: 0.0013 - val_loss: 0.0035
Epoch 4/100
192/192 [==============================] - 3s 18ms/step - loss: 0.0014 - val_loss: 0.0032
Epoch 5/100
192/192 [==============================] - 3s 16ms/step - loss: 0.0011 - val_loss: 0.0032
Epoch 6/100
192/192 [==============================] - 3s 17ms/step - loss: 0.0010 - val_loss: 0.0031
Epoch 7/100
192/192 [==============================] - 4s 18ms/step - loss: 0.0010 - val_loss: 0.0034
Epoch 8/100
192/192 [==============================] - 3s 16ms/step - loss: 8.9904e-04 - val_loss: 0.0105
Epoch 9/100
192/192 [==============================] - 3s 17ms/step - loss: 8.9921e-04 - val_loss: 0.0024
Epoch 10/100
192/192 [==============================] - 3s 16ms/step - los

In [ ]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

16/16 [==============================] - 0s 5ms/step


((957, 1), (508, 1))

In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="lstmevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
MSE_Test.append(mean_squared_error(original_ytest,test_predict))
MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.0227858078204948
Train data MSE:  4.0916624243200115
Test data MAE:  1.4484653888127874
-------------------------------------------------------------------------------------
Test data RMSE:  4.176788829038478
Test data MSE:  17.445564922380623
Test data MAE:  3.2659958963416207


<a name="lstmevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9963309105999046
Test data explained variance regression score: 0.9750143298570069


<a name="lstmrsquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9958928590226602
Test data R2 score: 0.9724260513897571


<a name="lstmrloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0004775140395663396
Test data MGD:  0.0008416113572520256
----------------------------------------------------------------------
Train data MPD:  0.04125197903665421
Test data MPD:  0.11905630759884443


<a name="lstmcomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="lstmpred10"></a>

#### Predicting next 10 days

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
               
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


<a name="lstmlast25"></a>

#### Plotting last 15 days and next predicted 10 days

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="lstmwholepred"></a>

#### Plotting whole closing stock price with prediction

In [ ]:
lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(lstmdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="gru"></a>

### GRU

In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (957, 15, 1)
X_test:  (508, 15, 1)



#### GRU model structure

In [ ]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(GRU(32,return_sequences=True,input_shape=(time_step,1)))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 15, 32)            3360      
                                                                 
 gru_1 (GRU)                 (None, 15, 32)            6336      
                                                                 
 gru_2 (GRU)                 (None, 15, 32)            6336      
                                                                 
 gru_3 (GRU)                 (None, 32)                6336      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 22,401
Trainable params: 22,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=5,verbose=1)

Epoch 1/200
192/192 [==============================] - 12s 29ms/step - loss: 0.0052 - val_loss: 0.0023
Epoch 2/200
192/192 [==============================] - 4s 23ms/step - loss: 9.3786e-04 - val_loss: 0.0027
Epoch 3/200
192/192 [==============================] - 5s 25ms/step - loss: 7.1115e-04 - val_loss: 0.0019
Epoch 4/200
192/192 [==============================] - 4s 23ms/step - loss: 5.6457e-04 - val_loss: 0.0014
Epoch 5/200
192/192 [==============================] - 5s 25ms/step - loss: 5.7883e-04 - val_loss: 0.0011
Epoch 6/200
192/192 [==============================] - 5s 24ms/step - loss: 4.9335e-04 - val_loss: 0.0017
Epoch 7/200
192/192 [==============================] - 5s 24ms/step - loss: 5.5526e-04 - val_loss: 8.6278e-04
Epoch 8/200
192/192 [==============================] - 5s 26ms/step - loss: 4.7055e-04 - val_loss: 0.0010
Epoch 9/200
192/192 [==============================] - 4s 23ms/step - loss: 3.4287e-04 - val_loss: 8.5752e-04
Epoch 10/200
192/192 [===================

In [ ]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

16/16 [==============================] - 0s 6ms/step


((957, 1), (508, 1))

In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="gruevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
MSE_Test.append(mean_squared_error(original_ytest,test_predict))
MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.565194831022332
Train data MSE:  6.580224521103689
Test data MAE:  1.896184903160521
-------------------------------------------------------------------------------------
Test data RMSE:  5.651641710624567
Test data MSE:  31.94105402527138
Test data MAE:  4.634259202014602


<a name="gruevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9949266766121778
Test data explained variance regression score: 0.9729754169889014


<a name="grursquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9933948828207126
Test data R2 score: 0.9495149061570407


<a name="grurloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0006832469418718605
Test data MGD:  0.0014122654214769898
----------------------------------------------------------------------
Train data MPD:  0.0632245386601098
Test data MPD:  0.20922766375137836


<a name="grucomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)


names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="grupred10"></a>

#### Predicting next 10 days

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
               
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


<a name="grulast25"></a>

#### Plotting last 15 days and next predicted 10 days

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})
names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="gruwholepred"></a>

#### Plotting whole closing stock price with prediction

In [ ]:
grudf=closedf.tolist()
grudf.extend((np.array(lst_output).reshape(-1,1)).tolist())
grudf=scaler.inverse_transform(grudf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])
fig = px.line(grudf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="LSTM+GRU"></a>

### LSTM + GRU

In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (957, 15, 1)
X_test:  (508, 15, 1)


#### Model structure

In [ ]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            8320      
                                                                 
 gru (GRU)                   (None, 15, 32)            6336      
                                                                 
 gru_1 (GRU)                 (None, 32)                6336      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 25,377
Trainable params: 25,377
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=5,verbose=1)

Epoch 1/200
192/192 [==============================] - 13s 34ms/step - loss: 0.0088 - val_loss: 0.0042
Epoch 2/200
192/192 [==============================] - 5s 23ms/step - loss: 0.0016 - val_loss: 0.0035
Epoch 3/200
192/192 [==============================] - 4s 22ms/step - loss: 0.0016 - val_loss: 0.0045
Epoch 4/200
192/192 [==============================] - 5s 25ms/step - loss: 0.0013 - val_loss: 0.0035
Epoch 5/200
192/192 [==============================] - 4s 22ms/step - loss: 0.0011 - val_loss: 0.0034
Epoch 6/200
192/192 [==============================] - 4s 23ms/step - loss: 0.0012 - val_loss: 0.0030
Epoch 7/200
192/192 [==============================] - 5s 23ms/step - loss: 0.0012 - val_loss: 0.0032
Epoch 8/200
192/192 [==============================] - 4s 22ms/step - loss: 8.9651e-04 - val_loss: 0.0027
Epoch 9/200
192/192 [==============================] - 5s 25ms/step - loss: 8.5563e-04 - val_loss: 0.0022
Epoch 10/200
192/192 [==============================] - 4s 22ms/step - lo

In [ ]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

16/16 [==============================] - 0s 6ms/step


((957, 1), (508, 1))

In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

<a name="lstm_gruevalmat"></a>

#### Evaluation metrices RMSE, MSE and MAE

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
RMSE_Train.append(math.sqrt(mean_squared_error(original_ytrain,train_predict)))
MSE_Train.append(mean_squared_error(original_ytrain,train_predict))
MAE_Train.append(mean_absolute_error(original_ytrain,train_predict))

RMSE_Test.append(math.sqrt(mean_squared_error(original_ytest,test_predict)))
MSE_Test.append(mean_squared_error(original_ytest,test_predict))
MAE_Test.append(mean_absolute_error(original_ytest,test_predict))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.044259266057831
Train data MSE:  4.178995946863303
Test data MAE:  1.511223364977591
-------------------------------------------------------------------------------------
Test data RMSE:  4.41415056318527
Test data MSE:  19.48472519446884
Test data MAE:  3.4096579201233395


<a name="lstm_gruevariance"></a>

#### Explained variance regression score


The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9960185087985088
Test data explained variance regression score: 0.9711020036819


<a name="lstm_grursquare"></a>

#### R<sup>2</sup> score for regression

R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

1 = Best <br>
0 or < 0 = worse

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9958051951217967
Test data R2 score: 0.9692030144287427


<a name="lstm_grurloss"></a>

#### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [ ]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0006240644642814943
Test data MGD:  0.0009213471640866622
----------------------------------------------------------------------
Train data MPD:  0.046668586490115495
Test data MPD:  0.1316844663102258


<a name="lstm_grucomparechart"></a>

#### Comparision between original stock close price vs predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1497, 1)
Test predicted data:  (1497, 1)


<a name="lstm_grupred10"></a>

#### Predicting next 10 days

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
               
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


<a name="lstm_grulast25"></a>

#### Plotting last 15 days and next predicted 10 days

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})
names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<a name="lstm_gruwholepred"></a>

#### Plotting whole closing stock price with prediction

In [ ]:
lstmgrudf=closedf.tolist()
lstmgrudf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmgrudf=scaler.inverse_transform(lstmgrudf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(lstmgrudf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
finaldf = pd.DataFrame({
    'svr':svrdf,
    'rf':rfdf,
    'knn':knndf,
    'lstm':lstmdf,
    'gru':grudf,
    'lstm_gru':lstmgrudf,
})
finaldf.head()

,svr,rf,knn,lstm,gru,lstm_gru
0,37.683498,37.683498,37.683498,37.683498,37.683498,37.683498
1,37.859001,37.859001,37.859001,37.859001,37.859001,37.859001
2,39.022499,39.022499,39.022499,39.022499,39.022499,39.022499
3,39.799500,39.799500,39.799500,39.799500,39.799500,39.799500
4,39.846001,39.846001,39.846001,39.846001,39.846001,39.846001


<a name="final_chart"></a>

## Conclusion Chart

In [ ]:
names = cycle(['SVR', 'RF','KNN','LSTM','GRU','LSTM + GRU'])

fig = px.line(finaldf[225:], x=finaldf.index[225:], y=[finaldf['svr'][225:],finaldf['rf'][225:], finaldf['knn'][225:], 
                                          finaldf['lstm'][225:], finaldf['gru'][225:], finaldf['lstm_gru'][225:]],
             labels={'x': 'Timestamp','value':'Stock close price'})
fig.update_layout(title_text='Final stock analysis chart', font_size=15, font_color='black',legend_title_text='Algorithms')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

## Evaluation Charts

### RMSE Training Error across all Models

In [ ]:
# Manually load evaluation metrics
RMSE_Train = [6.396151196361093, 0.7807095843938214, 3.333577551567551, 2.4643188619547325, 2.1837192718530325, 2.5275353860529775, 2.140876606687836, 2.11730300048861, 2.147169386804518, 2.0292274212694608, 1.9645336062778445]
MSE_Train = [40.91075012671144, 0.6095074551643734, 11.112739292315108, 6.072867453385869, 4.768629858262338, 6.388435127749974, 4.583352645063224, 4.482971995878072, 4.6103363756304905, 4.117763927231906, 3.8593922901950326]
MAE_Train = [4.9349600925875325, 0.5084457758111793, 2.586193950814241, 1.6150384879818878, 1.371088570402539, 1.5981650865003294, 1.3792536063608223, 1.3750842279453042, 1.5518825971419583, 1.450423468209431, 1.3711154416909943]
RMSE_Test = [8.62121192472498, 6.911474692499186, 7.356576061245299, 9.72230722448106, 3.5974891853500224, 4.406223800035478, 3.700723691573236, 3.6936644767797824, 3.7404720179785897, 4.162835209957822, 3.6817312973485543]
MSE_Test = [74.3252950510202, 47.76848242505672, 54.1192113448874, 94.52325776719661, 12.941928438710367, 19.414808175999088, 13.69535584137144, 13.643157267024863, 13.991130917280822, 17.329196985264588, 13.555145345875868]
MAE_Test = [6.758134014532531, 4.862012644106501, 5.484966800178278, 7.5885011683136465, 2.5954411293909283, 3.128677483088571, 2.6715130855491482, 2.674299753378765, 2.7604951775818938, 3.225707061114973, 2.760483387092652]

print(RMSE_Train)
print(MSE_Train)
print(MAE_Train)

print(RMSE_Test)
print(MSE_Test)
print(MAE_Test)

#
with open('train_eval_metrics.npy', 'wb') as f:
  np.save(f, RMSE_Train)
  np.save(f, MSE_Train)
  np.save(f, MAE_Train)


names = ['SVR', 'RF', 'SVR + RF', 'KNN', 'ARIMA', 'SARIMA', 'GRU + ARIMA', 'LSTM + ARIMA', 'LSTM','GRU','LSTM + GRU']

fig = px.line(x=names, y=RMSE_Train,
             labels={'x': 'Model','value':'RMSE Train'})
fig.update_layout(title_text='RMSE Train Error across Models', font_size=15, font_color='black',legend_title_text='Algorithms')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

[6.396151196361093, 0.7807095843938214, 3.333577551567551, 2.4643188619547325, 2.1837192718530325, 2.5275353860529775, 2.140876606687836, 2.11730300048861, 2.147169386804518, 2.0292274212694608, 1.9645336062778445]
[40.91075012671144, 0.6095074551643734, 11.112739292315108, 6.072867453385869, 4.768629858262338, 6.388435127749974, 4.583352645063224, 4.482971995878072, 4.6103363756304905, 4.117763927231906, 3.8593922901950326]
[4.9349600925875325, 0.5084457758111793, 2.586193950814241, 1.6150384879818878, 1.371088570402539, 1.5981650865003294, 1.3792536063608223, 1.3750842279453042, 1.5518825971419583, 1.450423468209431, 1.3711154416909943]
[8.62121192472498, 6.911474692499186, 7.356576061245299, 9.72230722448106, 3.5974891853500224, 4.406223800035478, 3.700723691573236, 3.6936644767797824, 3.7404720179785897, 4.162835209957822, 3.6817312973485543]
[74.3252950510202, 47.76848242505672, 54.1192113448874, 94.52325776719661, 12.941928438710367, 19.414808175999088, 13.69535584137144, 13.6431

In [ ]:
names = ['SVR', 'RF', 'SVR + RF', 'KNN', 'ARIMA', 'SARIMA', 'GRU + ARIMA', 'LSTM + ARIMA', 'LSTM','GRU','LSTM + GRU']
RMSE_Test = [8.62121192472498, 6.911474692499186, 7.356576061245299, 9.72230722448106, 3.5974891853500224, 4.406223800035478, 3.700723691573236, 3.6936644767797824, 3.7404720179785897, 4.162835209957822, 3.6817312973485543]
RMSE_Train = [6.396151196361093, 0.7807095843938214, 3.333577551567551, 2.4643188619547325, 2.1837192718530325, 2.5275353860529775, 2.140876606687836, 2.11730300048861, 2.147169386804518, 2.0292274212694608, 1.9645336062778445]

result1, result2 = zip(*sorted(zip(names, RMSE_Train), key=lambda x: x[1]))
print(result1)
print(result2)


('RF', 'LSTM + GRU', 'GRU', 'LSTM + ARIMA', 'GRU + ARIMA', 'LSTM', 'ARIMA', 'KNN', 'SARIMA', 'SVR + RF', 'SVR')
(0.7807095843938214, 1.9645336062778445, 2.0292274212694608, 2.11730300048861, 2.140876606687836, 2.147169386804518, 2.1837192718530325, 2.4643188619547325, 2.5275353860529775, 3.333577551567551, 6.396151196361093)


### RMSE Testing Error across all Models

In [ ]:
names = ['SVR', 'RF', 'SVR + RF', 'KNN', 'ARIMA', 'SARIMA', 'GRU + ARIMA', 'LSTM + ARIMA', 'LSTM','GRU','LSTM + GRU']

fig = px.line(x=names, y=RMSE_Test,
             labels={'x': 'Model','value':'RMSE Train'})
fig.update_layout(title_text='RMSE Test Error across Models', font_size=15, font_color='black',legend_title_text='Algorithms')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

### MSE Train Error

In [ ]:
names = ['SVR', 'RF', 'SVR + RF', 'KNN', 'ARIMA', 'SARIMA', 'GRU + ARIMA', 'LSTM + ARIMA', 'LSTM','GRU','LSTM + GRU']

fig = px.line(x=names, y=MSE_Train,
             labels={'x': 'Model','value':'MSE Train'})
fig.update_layout(title_text='MSE Train Error across Models', font_size=15, font_color='black',legend_title_text='Algorithms')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

### MSE Test Error

In [ ]:
names = ['SVR', 'RF', 'SVR + RF', 'KNN', 'ARIMA', 'SARIMA', 'GRU + ARIMA', 'LSTM + ARIMA', 'LSTM','GRU','LSTM + GRU']

fig = px.line(x=names, y=MSE_Test,
             labels={'x': 'Model','value':'MSE Test'})
fig.update_layout(title_text='MSE Test Error across Models', font_size=15, font_color='black',legend_title_text='Algorithms')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

### MAE Train Error

In [ ]:
names = ['SVR', 'RF', 'SVR + RF', 'KNN', 'ARIMA', 'SARIMA', 'GRU + ARIMA', 'LSTM + ARIMA', 'LSTM','GRU','LSTM + GRU']

fig = px.line(x=names, y=MAE_Train,
             labels={'x': 'Model','value':'MAE Train'})
fig.update_layout(title_text='MAE Train Error across Models', font_size=15, font_color='black',legend_title_text='Algorithms')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

### MAE Test Error

In [ ]:
names = ['SVR', 'RF', 'SVR + RF', 'KNN', 'ARIMA', 'SARIMA', 'GRU + ARIMA', 'LSTM + ARIMA', 'LSTM','GRU','LSTM + GRU']

fig = px.line(x=names, y=MAE_Test,
             labels={'x': 'Model','value':'MAE Test'})
fig.update_layout(title_text='MAE Test Error across Models', font_size=15, font_color='black',legend_title_text='Algorithms')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()